In [68]:
from queue import PriorityQueue
from copy import deepcopy

In [69]:
class Library:
    def __init__(self, id, n_books, signup_time, books_per_day, books):
        self.id = id
        self.n_books = n_books                  # number of books
        self.signup_time = signup_time          # signup time
        self.books_per_day = books_per_day      # books per day
        self.books = books
    
    def __str__(self):
        return f"{self.id} {self.n_books} {self.signup_time} {self.books_per_day} {self.books}"
    
    def calculate_score_per_day(self):
        score = 0
        green_books = 0
        pq = PriorityQueue()
        while (not self.books.empty()):
            book = self.books.get()
            if (book.status == 0):
                score += book.score
                green_books += 1
                pq.put(book)
            else:
                break
        
        while (not pq.empty()):
            self.books.put(pq.get())
        
        time_taken = self.signup_time + green_books / self.books_per_day

        return (score * self.books_per_day / (green_books * time_taken), green_books)
        

    def __lt__(self, other):
        spd1 = self.calculate_score_per_day()
        spd2 = other.calculate_score_per_day()
        if (spd1[0] == spd2[0]):
            return spd1[1] > spd2[1]
        return spd1[0] > spd2[0]

class Book:
    def __init__(self, id, score):
        self.id = id
        self.score = score
        self.status = 0
    
    def __str__(self):
        return f"{self.id} {self.score}" 
    
    def __lt__(self, other):
        if (self.status == other.status):
            return self.score > other.score
        return self.status < other.status
    
    def turn_yellow(self):
        self.status = 1

    def turn_red(self):
        self.status = 2
    
class Singleton:
    def __init__(self, total_books, total_libs, total_days, scores, libs):
        self.total_books = total_books  # total number of books
        self.total_libs = total_libs  # total number of libraries
        self.total_days = total_days  # total number of days
        self.scores = scores
        self.libs = libs

    def __str__(self):
        return f"{self.total_books} {self.total_libs} {self.total_days} {self.libs}"


In [70]:
libs_dict = {}
books_dict = {}

def read_file(filename):

    f = open(filename, "r")

    line = f.readline()

    values = [int(x) for x in line.split()]

    total_books = values[0]
    total_libs = values[1]
    total_days = values[2]

    line = f.readline()
    scores = [int(x) for x in line.split()]

    for i in range(len(scores)):
        books_dict[i] = Book(i, scores[i])

    libs = PriorityQueue()
    i = 0
    while True:
        line = f.readline()    
        values = [int(x) for x in line.split()]

        if len(values) == 0:
            break
        
        line = f.readline()
        books = [int(x) for x in line.split()]

        q = PriorityQueue()
        for b in books:
            q.put(books_dict[b])

        lib = Library(i, values[0], values[1], values[2], q)
        
        libs.put(lib)
        libs_dict[i] = lib
        i += 1

    f.close()

    return total_books, total_libs, total_days, scores, libs

In [74]:
def calculate_score(solution, total_days):
    score = 0
    days = 0
    scanned_books = set()
    libs_signed_up = set()
    
    for tup in solution:
        lib = libs_dict[tup[0]]

        if (lib.id in libs_signed_up):
            return -1

        days += lib.signup_time

        if (days >= total_days):
            break

        days_left = total_days - days # 2
        books_left = days_left * lib.books_per_day # 4
        books = tup[1][:books_left]
        
        for b in books:
            if (b not in scanned_books):
                score += books_dict[b].score
                scanned_books.add(b)
        
        libs_signed_up.add(lib.id)
    
    return score


In [75]:
total_books, total_libs, total_days, scores, libs = read_file("data/a_example.txt")
#total_books, total_libs, total_days, scores, libs= read_file("data/b_read_on.txt")
#total_books, total_libs, total_days, scores, libs = read_file("data/c_incunabula.txt")
#total_books, total_libs, total_days, scores, libs = read_file("data/d_tough_choices.txt")
#total_books, total_libs, total_days, scores, libs = read_file("data/e_so_many_books.txt")
#total_books, total_libs, total_days, scores, libs = read_file("data/f_libraries_of_the_world.txt")

att = Singleton(total_books, total_libs, total_days, scores, libs)

"""
while (not att.libs.empty()):
    lib = att.libs.get()
    print(lib.calculate_score_per_day())
    while not lib.books.empty():
        print(lib.books.get())
"""

print(calculate_score([(0, [3, 4, 2, 1, 0]), (1, [5, 3, 2, 0])], 7))

(1.511111111111111, 5)
3 6
4 5
2 3
1 2
0 1
(0.5, 4)
3 6
5 4
2 3
0 1
21
